In [ ]:
#hide
from neocov.core import *

# neocov

> social semantic variation

This file will become your README and also the index of your documentation.

## Install

`pip install your_project_name`

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2

In [ ]:
import say_hello

ModuleNotFoundError: No module named 'say_hello'

In [ ]:
import add

ModuleNotFoundError: No module named 'add'